# Dependency Extraction via include/import directive

## Steps
1. Recurse through each file of source, use the imports to determine dependency relations, generate TAs
2. Save the extracted dependencies to a raw.ta file
3. (Optional) Use pipeline to visualize extracted depedencies

In [22]:
# Step 1: Generate dependencies
import os

src = ".\\flink-1.17.1"
dependencies = []


def package_to_path(parent_module, package_name):
  return parent_module + package_name.replace(".", "\\") + ".java"


def process_line(file_path, line):
  # Example: 
  # line = import org.apache.flink.api.common.functions.MapFunction;
  # first need to remove the ; at end and split by space
  dependency = line.split(" ")[1].strip().replace(";", "")

  # Example:
  # file_path = .\flink-1.17.1\flink-core\src\main\java\org\apache\flink\api\common\functions\MapFunction.java
  # want to only get the parent directory up until /java/ at which point things will be scoped by package (org.apache.flink.*)
  parent_module = file_path.split("\\java\\")[0] + "\\java\\"

  # now we can convert the package to a path and add the tuple to our list
  dependency = package_to_path(parent_module, dependency)
  dependencies.append((file_path[2:].replace("\\", "/"), dependency[2:].replace("\\", "/")))


def process_file(file_path):
  with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
        if not line.startswith("import") or not "org.apache" in line:
            continue
        # this is a static import; not sure if we care about them or if we should just remove the static keyword
        # for now we will just ignore them
        if "static" in line: 
          continue
        process_line(file_path, line)


def generate_dependencies():
  for dir_path, _, file_names in os.walk(src):
    for file_name in file_names:
      if file_name.endswith(".java") and not "\\test\\" in dir_path:
        process_file(dir_path + "\\" + file_name)


generate_dependencies()
# print out the dependencies to verify they are correct

limit = 15
for file_path, dependency in dependencies:
  print(file_path + " -> " + dependency)

  limit -= 1
  if limit == 0:
    break

print(len(dependencies))


flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/FlinkVersion.java -> flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/Public.java
flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/docs/ConfigGroup.java -> flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/Internal.java
flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/docs/ConfigGroups.java -> flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/Internal.java
flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/docs/Documentation.java -> flink-1.17.1/flink-annotations/src/main/java/org/apache/flink/annotation/Internal.java
flink-1.17.1/flink-architecture-tests/flink-architecture-tests-production/src/main/java/org/apache/flink/architecture/ProductionCodeArchitectureBase.java -> flink-1.17.1/flink-architecture-tests/flink-architecture-tests-production/src/main/java/org/apache/flink/architecture

In [18]:
# Step 2: Generate the raw.ta file
raw_ta_output = "./visualize/source/dependencies.raw.ta"

with open(raw_ta_output, "w+") as f:
  # first generate all the concrete instances
  f.write("FACT TUPLE : \n")

  unique_file_paths = set(file_path for file_path, _ in dependencies)

  for file_path in unique_file_paths:
    f.write(f"$INSTANCE {file_path} cFile\n")

  # now add in all the dependencies
  for file_path, dependency in dependencies:
    f.write(f"cLinks {file_path} {dependency}\n")

In [19]:
# (Optional) Step 3: Run pipeline
!cd visualize && python ./pipeline.py

Successfully deleted flink_UnderstandFileDependency.raw.ta
Successfully deleted flink_UnderstandFileDependency.ls.ta
Successfully deleted flink_UnderstandFileDependency.contain
Create Containment Output:

c:\Users\k899e\projects\school\4314\a4\visualize>java -Xms256M -Xmx1024M -classpath ql.jar ca.uwaterloo.cs.ql.Main addcontain.ql flink_UnderstandFileDependency.contain flink_UnderstandFileDependency.raw.ta flink_UnderstandFileDependency.con.ta 
  Error[addcontain.ql]: extra files included in contain
     Channel.ss
     CheckpointCoordinator.ss
     Dispatcher.ss
     Entrypoint.ss
     ExecutionGraph.ss
     FlinkClients.ss
     Hooks.ss
     JobGraph.ss
     JobManager.ss
     JobMaster.ss
     Metadata.ss
     ResourceManager.ss
     Security.ss
     State.ss
     TaskExecutor.ss
     TaskManager.ss

c:\Users\k899e\projects\school\4314\a4\visualize>type schema.asv.ta flink_UnderstandFileDependency.con.ta  1>flink_UnderstandFileDependency.ls.ta 


Create Containment Errors:
The syst